In [1]:
import pandas as pd
import re
import numpy as np
import functions

In [2]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x0000027DD6DF2598; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


# Tabla Detalle

In [11]:
prueba = Detalle(cur)

In [12]:
prueba.to_excel('./detalle_cartera_20200123_excel_3.xlsx',encoding='CP1252',index=False)

# Base

In [4]:
def Cli_altas_bajas(df):
    df = df[df['rfb__c'].notnull()]  
    assets = df.isAsset.groupby(by=[df.cif,df.canal_venta]).sum().reset_index()
    assets.rename(columns={'isAsset':'assets'}, inplace=True)
    assets_act = df.isActive.groupby(by=[df.cif,df.canal_venta]).sum().reset_index()
    assets_act.rename(columns={'isActive':'assets_act'}, inplace=True)
    altas = df.rfb__c.groupby(by=[df.cif,df.canal_venta]).min().reset_index()
    bajas = df.deactivation_date.groupby(by=[df.cif,df.canal_venta]).max().reset_index()
    final = pd.merge(pd.merge(pd.merge(assets_act,altas,on=['cif','canal_venta']),bajas,on=['cif','canal_venta']),assets,on=['cif','canal_venta'])
    final['isClient'] = 1
    final['isActive'] = final.assets_act.apply(lambda x: 1 if x>=1 else 0)
    final['deactivation_date'] = final.apply(lambda row: functions.deactivation(row), axis = 1)
    final['assets_dea'] = final['assets'] - final['assets_act']
    final['isBaja'] = -(final['isActive']-1)
    final.rename(columns={'assets':'assets_tot',
                      'rfb__c':'rfb',
                      }, inplace = True)
    return final

def Cli_altas_bajas_mes(altas_bajas_df):
    altas = altas_bajas_df[['isClient']].groupby(by=[altas_bajas_df.rfb]).sum()
    bajas = altas_bajas_df[['isBaja']].groupby(by=[altas_bajas_df.deactivation_date]).sum()
    bajas.index.name = 'rfb'
    altas_bajas = altas.join(bajas, how='outer').fillna(0.0)
    mes = altas_bajas.groupby(by=[altas_bajas.index.year, altas_bajas.index.month]).sum()
    mes.index.names = ['year', 'month']
#     mes.reset_index(inplace=True)
    return mes

def Cli_altas_bajas_mes_canal(altas_bajas_df):
    altas = altas_bajas_df[['isClient']].groupby(by=[altas_bajas_df.rfb, altas_bajas_df.canal_venta]).sum()
    bajas = altas_bajas_df[['isBaja']].groupby(by=[altas_bajas_df.deactivation_date,altas_bajas_df.canal_venta]).sum()
    bajas.index.names = ['rfb','canal_venta']
    altas_bajas = altas.join(bajas, how='outer').fillna(0.0)
    mes_canal = altas_bajas.groupby(by=[altas_bajas.index.get_level_values('rfb').year, altas_bajas.index.get_level_values('rfb').month, altas_bajas.index.get_level_values('canal_venta')]).sum()
    mes_canal.index.names = ['year', 'month', 'canal']
    mes_canal.reset_index(inplace=True)
    return mes_canal

In [3]:
df = functions.base(cur)

In [4]:
print('Filtas Cartera:', len(df))
print('Total ventas:', len(df[df['isSold']==1]))
print('Total cancelaciones:', len(df[df['isCancelled']==1]))
print('Total assets:', len(df[df['isAsset']==1]))
print('Total bajas:', len(df[df['isBaja']==1]))
print('Total assets activos:', len(df[df['isActive']==1]))

Filtas Cartera: 1218
Total ventas: 1218
Total cancelaciones: 38
Total assets: 1111
Total bajas: 217
Total assets activos: 894


In [5]:
df.head(2)

,segmento,assetid,cif,canal_venta,product_name,migrado,deactivation_date,cancellation_date,rfb__c,purchase_date__c,Family,Velocity,isNeba,isSold,isAsset,isBaja,isActive,isCancelled
1,3-NOTRIAL,02i0N000006M6bnQAC,B85204915,DHO,Assets,None,NaT,NaT,2019-02-12 00:00:01,2019-02-12 00:00:01,X MUSIC,,0,1,1,0,1,0
2,3-NOTRIAL,02i0N000006M6boQAC,B85204915,DHO,Assets,None,NaT,NaT,2019-02-12 00:00:01,2019-02-12 00:00:01,X MUSIC,,0,1,1,0,1,0


# VENTAS

In [4]:
functions.ass_vta_fam_mes(df)

purchase_year  2018         2019                                               \
purchase_month   10  11  12    1   2   3   4   5    6    7   8    9   10   11   
Family                                                                          
X FIBER           0   4   8    7  10   2   8  13   17   17   4   20   17    2   
X MUSIC           3   2   3    4   7   2   0   6    0    0   0    5    0    0   
X PRIVACY         0   5   4    0  10  12   3  12   13    6   2    9    6    0   
X PROTECTION      0   0   4    2   5  25  13   4   29   20   5   19   28   28   
X SECURITY        0   0   2    2   0   0   0   5   13    5   0   53   28    2   
X UCOM            0  13  11   20  22  11  16  22   65   57  12   59   94   78   
All               3  24  32   35  54  52  40  62  137  105  23  165  173  110   

purchase_year       2020   All  
purchase_month   12    1        
Family                          
X FIBER           3    7   139  
X MUSIC           0    1    33  
X PRIVACY         2   11    95  
X PROTECTION      4    1   187  
X SECURITY        2    1   113  
X UCOM          120   51   651  
All             131   72  1218

In [5]:
functions.ass_vta_prod_mes(df)

purchase_year                   2018         2019                            \
purchase_month                    10  11  12    1   2   3   4   5    6    7   
Family       product_name                                                     
X FIBER      X Fiber 100 - neba    0   0   0    1   2   0   1   2    0    1   
             X Fiber 100 - own     0   0   4    3   2   0   4   3   12    1   
             X Fiber 200 - neba    0   1   1    0   3   1   2   2    4    5   
             X Fiber 200 - own     0   0   1    0   0   0   0   6    0    2   
             X Fiber 50 - neba     0   1   1    1   3   0   0   0    0    4   
             X Fiber 50 - own      0   2   1    2   0   1   1   0    1    4   
X MUSIC      Assets                3   2   3    4   7   2   0   6    0    0   
X PRIVACY    X Privacy 100         0   0   0    0   2   0   2   0    3    0   
             X Privacy 200         0   0   1    0   0   0   0   4    5    1   
             X Privacy 50          0   5   3    0   8  12   1   8    5    5   
X PROTECTION Assets                0   0   4    2   5  25  13   4   29   20   
X SECURITY   X Security 100        0   0   2    2   0   0   0   5   11    2   
             X Security 200        0   0   0    0   0   0   0   0    2    3   
X UCOM       Teamwork+PBX          0  13  11   20  22  11  16  22   65   57   
All                                3  24  32   35  54  52  40  62  137  105   

purchase_year                                           2020   All  
purchase_month                    8    9   10   11   12    1        
Family       product_name                                           
X FIBER      X Fiber 100 - neba   1    1   13    0    1    0    23  
             X Fiber 100 - own    0   15    4    0    0    0    48  
             X Fiber 200 - neba   0    2    0    0    0    3    24  
             X Fiber 200 - own    1    2    0    1    2    2    17  
             X Fiber 50 - neba    1    0    0    1    0    0    12  
             X Fiber 50 - own     1    0    0    0    0    2    15  
X MUSIC      Assets               0    5    0    0    0    1    33  
X PRIVACY    X Privacy 100        0    0    2    0    2    0    11  
             X Privacy 200        0    4    4    0    0   10    29  
             X Privacy 50         2    5    0    0    0    1    55  
X PROTECTION Assets               5   19   28   28    4    1   187  
X SECURITY   X Security 100       0   50   27    0    2    0   101  
             X Security 200       0    3    1    2    0    1    12  
X UCOM       Teamwork+PBX        12   59   94   78  120   51   651  
All                              23  165  173  110  131   72  1218

In [6]:
functions.cli_vta_fam_mes(df,0)

purchase_year  2018         2019                                              \
purchase_month   10  11  12    1   2   3   4   5   6   7   8   9  10  11  12   
canal_venta                                                                    
DHO               2   6   7    9  22  13   7  15  34  43  12  35  40  36  34   
IT integrator     0   1   4    3   1   1   8   3   5   4   1   6   2   2   1   
WEB               1   4   7    1   1   7   4   7   6   5   3   9   7   4   3   
All               3  11  18   13  24  21  19  25  45  52  16  50  49  42  38   

purchase_year  2020  All  
purchase_month    1       
canal_venta               
DHO              21  336  
IT integrator     1   43  
WEB               4   73  
All              26  452

# Altas

In [18]:
df2 = Cli_altas_bajas(df)

In [19]:
df2.head()

,cif,canal_venta,assets_act,rfb,deactivation_date,assets_tot,isClient,isActive,assets_dea,isBaja
0,03817373V,WEB,0,2019-11-20 00:00:01,2019-11-20 15:29:08,2,1,0,2,1
1,07477239P,DHO,0,2018-10-25 09:38:28,2019-01-21 09:13:32,1,1,0,1,1
2,11058321J,DHO,1,2019-01-23 00:00:00,NaT,1,1,1,0,0
3,12371541W,WEB,0,2018-12-18 00:00:01,2019-01-02 12:15:09,1,1,0,1,1
4,16601899Q,DHO,1,2019-05-22 00:00:01,NaT,5,1,1,4,0


In [23]:
a = Cli_altas_bajas_mes_canal(df2)

In [24]:
a[((a['year']==2019)|(a['year']==2020))&((a['month']==12)|(a['month']==1))]

,year,month,canal,isClient,isBaja
8,2019,1,DHO,7.0,1.0
9,2019,1,IT integrator,3.0,0.0
10,2019,1,WEB,3.0,2.0
38,2019,12,DHO,24.0,6.0
39,2019,12,WEB,1.0,1.0
40,2020,1,DHO,22.0,7.0
41,2020,1,WEB,3.0,0.0
